### Training RL Policies using L5Kit Closed-Loop Environment

This notebook describes how to train RL policies for self-driving using our gym-compatible closed-loop environment.

We will be using [Proximal Policy Optimization (PPO)](https://arxiv.org/abs/1707.06347) algorithm as our reinforcement learning algorithm, as it not only demonstrates remarkable performance but it is also empirically easy to tune.

The PPO implementation in this notebook is based on [Stable Baselines3](https://github.com/DLR-RM/stable-baselines3) framework, a popular framework for training RL policies. Note that our environment is also compatible with [RLlib](https://docs.ray.io/en/latest/rllib.html), another popular frameworks for the same.

ref: 
([rllib] Best workflow to train, save, and test agent #9123
)[https://github.com/ray-project/ray/issues/9123]

In [1]:
import os
os.environ["L5KIT_DATA_FOLDER"] = '/DATA/l5kit/l5kit-dataset'
# os.environ["TUNE_RESULT_DIR"] =  '/DATA/l5kit/rllib_tb_logs'

In [2]:
import gym

# from stable_baselines3 import PPO
# from stable_baselines3.common.callbacks import CheckpointCallback
# from stable_baselines3.common.env_util import make_vec_env
# from stable_baselines3.common.utils import get_linear_fn
# from stable_baselines3.common.vec_env import SubprocVecEnv

from l5kit.configs import load_config_data
# from l5kit.environment.feature_extractor import CustomFeatureExtractor
# from l5kit.environment.callbacks import L5KitEvalCallback
from l5kit.environment.envs.l5_env import SimulationConfigGym, GymStepOutput, L5Env

from l5kit.visualization.visualizer.zarr_utils import episode_out_to_visualizer_scene_gym_cle
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.environment.gym_metric_set import L2DisplacementYawMetricSet, CLEMetricSet
from prettytable import PrettyTable
import datetime
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
import torch.nn as nn
import numpy as np
import gym
from typing import Dict
import numpy as np
import ray
import pytz


## Init ray and env

In [4]:
# Dataset is assumed to be on the folder specified
# in the L5KIT_DATA_FOLDER environment variable
from l5kit.configs import load_config_data

# get environment config
env_config_path = '/DATA/l5kit/configs/gym_config84.yaml'
cfg = load_config_data(env_config_path)


In [5]:
newYorkTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(newYorkTz).strftime("%d-%m-%Y_%H-%M-%S")
ray_result_logdir = '/DATA/l5kit/ray_results/' + date

ray.init(num_cpus=64, ignore_reinit_error=True, log_to_driver=False)

2023-01-01 08:30:22,200	WARNING services.py:1732 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67018752 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-01-01 08:30:23,356	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.2.0


## Customize my model
SAC: https://github.com/ray-project/ray/blob/dfb9689701361cfd18f383e0a3edeed6baf81abb/rllib/agents/sac/sac_torch_model.py

In [6]:
class GNCNN(SACTorchModel):
    """
    Simple Convolution agent that calculates the required linear output layer
    """

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        super().__init__(obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)

        # raise ValueError(obs_space.shape)
        self._num_objects = obs_space.shape[2] # num_of_channels of input, size x size x channels
        self._num_actions = num_outputs
        self._feature_dim = model_config["custom_model_config"]['feature_dim']

        # linear_flatten = np.prod(obs_space.shape[:2])*64

        self.network = nn.Sequential(
            nn.Conv2d(self._num_objects, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.GroupNorm(4, 64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
            nn.GroupNorm(2, 32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(in_features=1568, out_features=self._feature_dim),
            # layer_init(nn.Conv2d(self._num_objects, 32, 3, padding=1)),
            # nn.ReLU(),
            # layer_init(nn.Conv2d(32, 64, 3, padding=1)),
            # nn.ReLU(),
            # nn.Flatten(),
            # layer_init(nn.Linear(linear_flatten, 1024)),
            # nn.ReLU(),
            # layer_init(nn.Linear(1024, 512)),
            # nn.ReLU(),
        )

        self._actor_head = nn.Sequential(
            # layer_init(nn.Linear(512, 256), std=0.01),
            # nn.ReLU(),
            # layer_init(nn.Linear(256, self._num_actions), std=0.01)
            nn.Linear(self._feature_dim, 256),
            nn.ReLU(),
            nn.Linear(256, self._num_actions),
        )

        self._critic_head = nn.Sequential(
            # layer_init(nn.Linear(512, 1), std=0.01)
            nn.Linear(self._feature_dim, 1),
        )

    def forward(self, input_dict, state, seq_lens):
        obs_transformed = input_dict['obs'].permute(0, 3, 1, 2) # 32 x 112 x 112 x 7 [B, size, size, channels]
        network_output = self.network(obs_transformed)
        value = self._critic_head(network_output)
        self._value = value.reshape(-1)
        logits = self._actor_head(network_output)
        return logits, state

    def value_function(self):
        return self._value
    def get_policy_output(self, , input_dict, state, seq_lens):
        return forward()
    def 

SyntaxError: invalid syntax (3054087535.py, line 63)

In [ ]:
from ray.rllib.models import ModelCatalog
ModelCatalog.register_custom_model(
        "GN_CNN_torch_model", GNCNN
    )

## Define Training and Evaluation Environments

**Training**: We will be training the PPO policy on episodes of length 32 time-steps. We will have 4 sub-processes (training environments) that will help to parallelize and speeden up episode rollouts. The *SimConfig* dataclass will define the parameters of the episode rollout: like length of episode rollout, whether to use log-replayed agents or simulated agents etc.

**Evaluation**: We will evaluate the performance of the PPO policy on the *entire* scene (~248 time-steps).

## Customize gym env

In [6]:
class L5EnvWrapper(gym.Wrapper):
    def __init__(self, env, raster_size = 112, n_channels = 7):
        super().__init__(env)
        self.env = env
        self.n_channels = n_channels
        self.raster_size = raster_size
        obs_shape = (self.raster_size, self.raster_size, self.n_channels)
        self.observation_space =gym.spaces.Box(low=0, high=1, shape=obs_shape, dtype=np.float32)

    def step(self, action:  np.ndarray) -> GymStepOutput:
        # return GymStepOutput(obs, reward["total"], done, info)
        output =  self.env.step(action)
        onlyImageState = output.obs['image'].reshape(self.raster_size, self.raster_size, self.n_channels)
        return GymStepOutput(onlyImageState, output.reward, output.done, output.info)

    def reset(self) -> Dict[str, np.ndarray]:
        return self.env.reset()['image'].reshape(self.raster_size, self.raster_size, self.n_channels)

In [7]:
from ray import tune
train_eps_length = 32
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
# Register , how your env should be constructed (always with 5, or you can take values from the `config` EnvContext object):
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}

tune.register_env("L5-CLE-V0", lambda config: L5Env(**env_kwargs))
tune.register_env("L5-CLE-V1", lambda config: L5EnvWrapper(env = L5Env(**env_kwargs), \
                                                           raster_size= cfg['raster_params']['raster_size'][0], \
                                                           n_channels = 7))

## Train

import numpy as np

import ray
from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback
os.environ['WANDB_NOTEBOOK_NAME'] = '/DATA/rllib_ppo_policy_training.ipynb'!wandb login 083592c84134c040dcca598c644c348d32540a08
import wandb ## ref

Resume stop tune: https://docs.ray.io/en/latest/tune/tutorials/tune-stopping.html

tune.Tuner analysis: https://docs.ray.io/en/latest/rllib/rllib-training.html#basic-python-api

get best result, load from dir: https://docs.ray.io/en/master/tune/examples/tune_analyze_results.html#trial-level-analysis-working-with-an-individual-result

In [8]:
!wandb login 083592c84134c040dcca598c644c348d32540a08

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
import numpy as np

import ray
from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback
os.environ['WANDB_NOTEBOOK_NAME'] = '/DATA/rllib_sac_policy_training.ipynb'

In [11]:
import wandb
wandb.init(project="l5kit2", reinit = True)

In [ ]:
import ray
from ray import air, tune
train_envs = 4
ray_result_logdir = '/DATA/l5kit/ray_results/' + date

config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    'q_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'policy_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'tau': 0.005,
    'target_network_update_freq': 5,
    'replay_buffer_config':{
        'type': 'PrioritizedReplayBuffer',
        'capacity': int(4e5),
    },
    'num_steps_sampled_before_learning_starts': 1000,
    
    'target_entropy': 'auto',
#     "model": {
#         "custom_model": "GN_CNN_torch_model",
#         "custom_model_config": {'feature_dim':128},
#     },
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
    # 'train_batch_size': 4000,
    # 'sgd_minibatch_size': 256,
    # 'num_sgd_iter': 16,
    'store_buffer_in_checkpoints' : True,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 32,
    'gamma': 0.8,
}

result_grid = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(6e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=2, checkpoint_frequency = 10, checkpoint_score_attribute = 'episode_reward_mean')
        ),
    param_space=config_param_space).fit()

In [ ]:
import ray
from ray import air, tune
train_envs = 4

hcmTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(hcmTz).strftime("%d-%m-%Y_%H-%M-%S")
ray_result_logdir = '/DATA/l5kit/ray_results/' + date

lr = 3e-3
lr_start = 3e-4
lr_end = 3e-5
config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    'q_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'policy_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'tau': 0.005,
    'target_network_update_freq': 1,
    'replay_buffer_config':{
        'type': 'MultiAgentPrioritizedReplayBuffer',
        'capacity': int(1e5),
        "worker_side_prioritization": True,
    },
    'num_steps_sampled_before_learning_starts': 8000,
    
    'target_entropy': 'auto',
#     "model": {
#         "custom_model": "GN_CNN_torch_model",
#         "custom_model_config": {'feature_dim':128},
#     },
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
 
    # 'train_batch_size': 4000,
    # 'sgd_minibatch_size': 256,
    # 'num_sgd_iter': 16,
    # 'store_buffer_in_checkpoints' : False,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 1,
    'train_batch_size': 2048,
    'training_intensity' : 32, # (4x 'natural' value = 8)
    'gamma': 0.8,
    'twin_q' : True,
    "lr": 3e-4,
    "min_sample_timesteps_per_iteration": 8000,
}

result_grid = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(4e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=2, checkpoint_frequency = 10, checkpoint_score_attribute = 'episode_reward_mean'),
        callbacks=[WandbLoggerCallback(project="l5kit2", 
						save_checkpoints=True),],
        ),
        
    param_space=config_param_space).fit()

2023-01-01 08:53:50,174	INFO wandb.py:250 -- Already logged into W&B.


Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
SAC_L5-CLE-V1_cf7bb_00000,80640,"{'num_env_steps_sampled': 80640, 'num_env_steps_trained': 2367488, 'num_agent_steps_sampled': 80640, 'num_agent_steps_trained': 2367488, 'last_target_update_ts': 80640, 'num_target_updates': 289}",{},2023-01-01_09-41-41,False,31,{},-58.2638,-266.552,-395.789,252,2520,728aa68f10b24a71807ff882bf21f6b9,38d1f3f62321,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'actor_loss': 8.812812805175781, 'critic_loss': 0.39401447772979736, 'alpha_loss': -1.7508076429367065, 'alpha_value': 0.7067752, 'log_alpha_value': -0.34704265, 'target_entropy': -3.0, 'policy_t': 0.022778328508138657, 'mean_q': -8.996729850769043, 'max_q': 0.1875818520784378, 'min_q': -19.785879135131836}, 'model': {}, 'num_grad_updates_lifetime': 1156.0, 'diff_num_grad_updates_vs_sampler_policy': 1155.0, 'td_error': array([6.3309903, 3.6437593, 4.47351 , ..., 4.5698147, 3.6771932, 1.9372492], dtype=float32), 'mean_td_error': 3.2561662197113037}}, 'num_env_steps_sampled': 80640, 'num_env_steps_trained': 2367488, 'num_agent_steps_sampled': 80640, 'num_agent_steps_trained': 2367488, 'last_target_update_ts': 80640, 'num_target_updates': 289}",10,172.17.0.2,80640,2367488,80640,8064,2367488,262144,0,63,0,0,262144,"{'cpu_util_percent': 21.818246445497632, 'ram_util_percent': 45.19928909952607}",1654720,{},{},{},"{'mean_raw_obs_processing_ms': 543.2517422664438, 'mean_inference_ms': 38.52429465642051, 'mean_action_processing_ms': 2.763415269503187, 'mean_env_wait_ms': 403.98261404789645, 'mean_env_render_ms': 0.0}",2813.26,306.02,2813.26,"{'training_iteration_time_ms': 9504.646, 'load_time_ms': 133.039, 'load_throughput': 15393.991, 'learn_time_ms': 164.86, 'learn_throughput': 12422.695, 'synch_weights_time_ms': 598.243}",1672566101,0,80640,10,cf7bb_00000,53.376


wandb: Adding directory to artifact (/DATA/l5kit/ray_results/01-01-2023_15-53-49/SAC/SAC_L5-CLE-V1_cf7bb_00000_0_2023-01-01_08-53-50/checkpoint_000010)... Done. 0.3s


In [38]:
import ray
from ray import air, tune
train_envs = 4

hcmTz = pytz.timezone("Asia/Ho_Chi_Minh") 
date = datetime.datetime.now(hcmTz).strftime("%d-%m-%Y_%H-%M-%S")
ray_result_logdir = '/DATA/l5kit/ray_results/' + date

lr = 3e-3
lr_start = 3e-4
lr_end = 3e-5
config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    'q_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'policy_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'tau': 0.005,
    'target_network_update_freq': 1,
    'replay_buffer_config':{
        'type': 'MultiAgentPrioritizedReplayBuffer',
        'capacity': int(1e5),
        "worker_side_prioritization": True,
    },
    'num_steps_sampled_before_learning_starts': 8000,
    
    'target_entropy': 'auto',
#     "model": {
#         "custom_model": "GN_CNN_torch_model",
#         "custom_model_config": {'feature_dim':128},
#     },
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
 
    # 'train_batch_size': 4000,
    # 'sgd_minibatch_size': 256,
    # 'num_sgd_iter': 16,
    # 'store_buffer_in_checkpoints' : False,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 1,
    'train_batch_size': 2048,
    # 'training_intensity' : 1000,
    'gamma': 0.8,
    'twin_q' : True,
    "lr": 3e-4,
    "min_sample_timesteps_per_iteration": 8000,
}

result_grid = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(2e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=2, checkpoint_frequency = 10, checkpoint_score_attribute = 'episode_reward_mean')
        ),
    param_space=config_param_space).fit()

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
SAC_L5-CLE-V1_5af7a_00000,2007936,"{'num_env_steps_sampled': 2007936, 'num_env_steps_trained': 16254976, 'num_agent_steps_sampled': 2007936, 'num_agent_steps_trained': 16254976, 'last_target_update_ts': 2007936, 'num_target_updates': 7937}",{},2022-12-29_09-10-12,True,31,{},-4.97356,-48.418,-342.104,252,64764,6a1b193b49884b9783bc68502a9fba4d,38d1f3f62321,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'actor_loss': 10.66728687286377, 'critic_loss': 0.2743040919303894, 'alpha_loss': -8.968500137329102, 'alpha_value': 0.097858936, 'log_alpha_value': -2.3242283, 'target_entropy': -3.0, 'policy_t': 0.002813179511576891, 'mean_q': -10.056241989135742, 'max_q': -0.7284332513809204, 'min_q': -71.80377197265625}, 'model': {}, 'num_grad_updates_lifetime': 7937.0, 'diff_num_grad_updates_vs_sampler_policy': 7936.0, 'td_error': array([3.714016 , 0.5761843, 1.6428685, ..., 3.9594688, 2.1804185, 0.9676223], dtype=float32), 'mean_td_error': 2.5138769149780273}}, 'num_env_steps_sampled': 2007936, 'num_env_steps_trained': 16254976, 'num_agent_steps_sampled': 2007936, 'num_agent_steps_trained': 16254976, 'last_target_update_ts': 2007936, 'num_target_updates': 7937}",249,172.17.0.2,2007936,16254976,2007936,8064,16254976,65536,0,63,0,0,65536,"{'cpu_util_percent': 38.95, 'ram_util_percent': 64.37439024390243}",2100464,{},{},{},"{'mean_raw_obs_processing_ms': 557.5587640255825, 'mean_inference_ms': 47.593003390485485, 'mean_action_processing_ms': 2.9690678893112814, 'mean_env_wait_ms': 426.11047655216566, 'mean_env_render_ms': 0.0}",30097.2,119.19,30097.2,"{'training_iteration_time_ms': 3943.931, 'load_time_ms': 185.149, 'load_throughput': 11061.376, 'learn_time_ms': 210.285, 'learn_throughput': 9739.143, 'synch_weights_time_ms': 458.456}",1672305012,0,2007936,249,5af7a_00000,56.4943


2022-12-29 09:10:12,594	INFO tune.py:762 -- Total run time: 30170.23 seconds (30169.98 seconds for the tuning loop).


In [43]:
config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    'q_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'policy_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'tau': 0.005,
    'target_network_update_freq': 1,
    'replay_buffer_config':{
        'type': 'MultiAgentPrioritizedReplayBuffer',
        'capacity': int(1e5),
        "worker_side_prioritization": True,
    },
    'num_steps_sampled_before_learning_starts': 8000,
    
    'target_entropy': 'auto',
#     "model": {
#         "custom_model": "GN_CNN_torch_model",
#         "custom_model_config": {'feature_dim':128},
#     },
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
 
    # 'train_batch_size': 4000,
    # 'sgd_minibatch_size': 256,
    # 'num_sgd_iter': 16,
    # 'store_buffer_in_checkpoints' : False,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 1,
    'train_batch_size': 2048,
    # 'training_intensity' : 1000,
    'gamma': 0.8,
    'twin_q' : True,
    "lr": 3e-4,
    "min_sample_timesteps_per_iteration": 8000,
}

In [45]:
# config_param_space['stop']['timesteps_total'] = 3e-5
path_to_trained_agent_checkpoint = 'l5kit/ray_results/29-12-2022_07-47-22/SAC/SAC_L5-CLE-V1_5af7a_00000_0_2022-12-29_00-47-23/checkpoint_000249'
from ray.rllib.algorithms.sac import SAC
ray.tune.run(SAC, config=config_param_space, restore=path_to_trained_agent_checkpoint)

Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
SAC_L5-CLE-V1_f4f69_00000,2016000,"{'num_env_steps_sampled': 2016000, 'num_env_steps_trained': 16320512, 'num_agent_steps_sampled': 2016000, 'num_agent_steps_trained': 16320512, 'last_target_update_ts': 2016000, 'num_target_updates': 7969}",{},2022-12-29_14-59-58,False,31,{},-15.617,-137.817,-371.548,252,65016,6a1b193b49884b9783bc68502a9fba4d,38d1f3f62321,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'actor_loss': 4.579573631286621, 'critic_loss': 0.9118623733520508, 'alpha_loss': -9.704286575317383, 'alpha_value': 0.09693226, 'log_alpha_value': -2.3337429, 'target_entropy': -3.0, 'policy_t': -0.08073669672012329, 'mean_q': -3.388627767562866, 'max_q': -0.7259072065353394, 'min_q': -15.54305648803711}, 'model': {}, 'num_grad_updates_lifetime': 32.0, 'diff_num_grad_updates_vs_sampler_policy': 31.0, 'td_error': array([1.2998202, 8.56468 , 1.5235349, ..., 2.8340416, 3.6547613, 1.2337224], dtype=float32), 'mean_td_error': 4.344608306884766}}, 'num_env_steps_sampled': 2016000, 'num_env_steps_trained': 16320512, 'num_agent_steps_sampled': 2016000, 'num_agent_steps_trained': 16320512, 'last_target_update_ts': 2016000, 'num_target_updates': 7969}",1,172.17.0.2,2016000,16320512,2016000,8064,16320512,65536,0,63,0,0,65536,"{'cpu_util_percent': 60.70904977375565, 'ram_util_percent': 42.32488687782805}",137387,{},{},{},"{'mean_raw_obs_processing_ms': 691.7444644724308, 'mean_inference_ms': 41.91511855095612, 'mean_action_processing_ms': 3.431972257789366, 'mean_env_wait_ms': 584.0683852668678, 'mean_env_render_ms': 0.0}",157.784,157.784,30255,"{'training_iteration_time_ms': 7780.57, 'load_time_ms': 205.295, 'load_throughput': 9975.888, 'learn_time_ms': 239.504, 'learn_throughput': 8551.009, 'synch_weights_time_ms': 2729.168}",1672325998,0,2016000,250,f4f69_00000,54.8865


2022-12-29 15:00:51,943	WARNING tune.py:690 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2022-12-29 15:00:54,331	ERROR tune.py:758 -- Trials did not complete: [SAC_L5-CLE-V1_f4f69_00000]
2022-12-29 15:00:54,333	INFO tune.py:762 -- Total run time: 272.99 seconds (272.61 seconds for the tuning loop).
2022-12-29 15:00:54,333	WARNING tune.py:768 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


*** SIGTERM received at time=1672326111 on cpu 28 ***
PC: @     0x7f0aa3d5b46e  (unknown)  epoll_wait
    @     0x7f0aa3c7f090  (unknown)  (unknown)
[2022-12-29 15:01:51,032 E 1533378 1533378] logging.cc:361: *** SIGTERM received at time=1672326111 on cpu 28 ***
[2022-12-29 15:01:51,032 E 1533378 1533378] logging.cc:361: PC: @     0x7f0aa3d5b46e  (unknown)  epoll_wait
[2022-12-29 15:01:51,032 E 1533378 1533378] logging.cc:361:     @     0x7f0aa3c7f090  (unknown)  (unknown)


In [ ]:
import ray
from ray import air, tune
train_envs = 4
ray_result_logdir = '/DATA/l5kit/ray_results/' + date
lr = 3e-3
lr_start = 3e-4
lr_end = 3e-5

config_param_space = {
    "env": "L5-CLE-V1",
    "framework": "torch",
    "num_gpus": 1,
    "num_workers": 63,
    "num_envs_per_worker": train_envs,
    'q_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'policy_model_config' : {
            # "dim": 112,
            # "conv_filters" : [[64, [7,7], 3], [32, [11,11], 3], [32, [11,11], 3]],
            # "conv_activation": "relu",
            "post_fcnet_hiddens": [256],
            "post_fcnet_activation": "relu",
        },
    'tau': 0.005,
    'n_step': 1
    'target_network_update_freq': 5,
    'replay_buffer_config':{
        'type': 'MultiAgentPrioritizedReplayBuffer',
        'capacity': int(3e5),
    },
    'num_steps_sampled_before_learning_starts': 256,
    
    'target_entropy': 'auto',
#     "model": {
#         "custom_model": "GN_CNN_torch_model",
#         "custom_model_config": {'feature_dim':128},
#     },
    '_disable_preprocessor_api': True,
     "eager_tracing": True,
     "restart_failed_sub_environments": True,
    # 'train_batch_size': 4000,
    # 'sgd_minibatch_size': 256,
    # 'num_sgd_iter': 16,
    'seed': 42,
    'batch_mode': 'truncate_episodes',
    "rollout_fragment_length": 32,
    'gamma': 0.8,
    "lr_schedule": [
        [1e6, lr_start],
        [2e6, lr_end],
    ],
    "lr": lr,
}

result_grid = tune.Tuner(
    "SAC",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 0, 'timesteps_total': int(3e6)},
        local_dir=ray_result_logdir,
        checkpoint_config=air.CheckpointConfig(num_to_keep=2, checkpoint_frequency = 10, checkpoint_score_attribute = 'episode_reward_mean')
        ),
    param_space=config_param_space).fit()

In [39]:
import ray
from ray import air, tune
train_envs = 4
ray_result_logdir = '/DATA/l5kit/ray_results/29-12-2022_07-47-22'

tuner = tune.Tuner.restore(
    path=ray_result_logdir + '/SAC'
)
result = tuner.fit()

2022-12-29 14:46:39,383	INFO experiment_analysis.py:795 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.
2022-12-29 14:46:39,426	INFO trial_runner.py:688 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2022-12-29 14:46:39,427	INFO trial_runner.py:825 -- Using following checkpoint to resume: /DATA/l5kit/ray_results/29-12-2022_07-47-22/SAC/experiment_state-2022-12-29_00-47-22.json
2022-12-29 14:46:39,429	WARNING trial_runner.py:830 -- Attempting to resume experiment from /DATA/l5kit/ray_results/29-12-2022_07-47-22/SAC. This will ignore any new changes to the specification.
2022-12-29 14:46:39,436	INFO tune.py:653 -- TrialRunner resumed, ignoring new add_experiment but updating trial resources.


2022-12-29 14:46:39,596	INFO tune.py:762 -- Total run time: 0.17 seconds (0.00 seconds for the tuning loop).


In [ ]:
best_result = result.get_best_result(metric="episode_reward_mean", mode = 'max')
best_checkpoint = best_result.checkpoint
best_checkpoint

In [19]:
num_results = len(result)
print("Number of results:", num_results)

Number of results: 1


In [ ]:
result.get_best_result()

In [34]:
result.get_dataframe()

""


In [40]:
type(best_result)

ray.air.result.Result

*** SIGTERM received at time=1671780714 on cpu 2 ***
PC: @     0x7f4dbec0d46e  (unknown)  epoll_wait
    @     0x7f4dbeb31090  (unknown)  (unknown)
[2022-12-23 07:31:54,799 E 2700257 2700257] logging.cc:361: *** SIGTERM received at time=1671780714 on cpu 2 ***
[2022-12-23 07:31:54,799 E 2700257 2700257] logging.cc:361: PC: @     0x7f4dbec0d46e  (unknown)  epoll_wait
[2022-12-23 07:31:54,799 E 2700257 2700257] logging.cc:361:     @     0x7f4dbeb31090  (unknown)  (unknown)


In [39]:
result_df = best_result.metrics_dataframe() 
result_df[['episode_reward_mean']]

TypeError: 'NoneType' object is not callable

NOTE: Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`

2022-12-04 05:50:38,570	INFO experiment_analysis.py:795 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.

2022-12-04 05:50:39,684	INFO trial_runner.py:601 -- A local experiment checkpoint was found and will be used to restore the previous experiment state.
2022-12-04 05:50:39,687	INFO trial_runner.py:738 -- Using following checkpoint to resume: /content/drive/MyDrive/Colab Notebooks/l5kit/ray_results/PPO/experiment_state-2022-12-04_05-28-55.json

2022-12-04 05:50:39,710	WARNING trial_runner.py:743 -- Attempting to resume experiment from /content/drive/MyDrive/Colab Notebooks/l5kit/ray_results/PPO. This will ignore any new changes to the specification.

2022-12-04 05:50:40,703	INFO tune.py:668 -- TrialRunner resumed, ignoring new add_experiment but updating trial resources.

In [ ]:
train_envs=2
ray_result_logdir = '/content/drive/MyDrive/Colab Notebooks/l5kit/ray_results'
# Create the Trainer.
algo = ppo.PPO(
        env="L5-CLE-V1",
        config={
            "framework": "torch",
            "num_gpus": 1,
            "num_workers": 2,
            "num_envs_per_worker": train_envs,
            'num_sgd_iter': 5,
            'sgd_minibatch_size': 256,
            'num_cpus_per_worker': 0,  # This avoids running out of resources in the notebook environment when this cell is re-executed
            "model": {
                "custom_model": "GN_CNN_torch_model",
                "custom_model_config": {'feature_dim':128},
            },
            '_disable_preprocessor_api': True,
        },
        logger_creator=custom_log_creator(os.path.expanduser(ray_result_logdir), 'L5_PPO')
    )

2022-12-04 04:42:57,593	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-12-04 04:42:57,599	INFO algorithm.py:457 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-12-04 04:43:38,550	WARNING catalog.py:641 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2022-12-04 04:43:51,371	INFO trainable.py:164 -- Trainable.setup took 53.780 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-12-04 04:43:51,397	WARNING util.py:66 -- Install gputil for GPU system monitoring.


In [ ]:
checkpoint_path =  '/content/drive/MyDrive/Colab Notebooks/l5kit/rllib_logs/'+ str(datetime.date.today())
for i in range(1, 1000):
   # Perform one iteration of training the policy with PPO
   result = algo.train()
   print(pretty_print(result))

   if i % 10 == 0:
       checkpoint = algo.save(checkpoint_dir= checkpoint_path)
       print("checkpoint saved at", checkpoint)

In [ ]:
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/l5kit/rllib_logs/2022-12-02/checkpoint_000130'
config={
            "framework": "torch",
            "num_gpus": 1,
            "num_workers": 2,
            "num_envs_per_worker": train_envs,
            'num_sgd_iter': 15,
            'sgd_minibatch_size': 128,
            'num_cpus_per_worker': 0,  # This avoids running out of resources in the notebook environment when this cell is re-executed
            "model": {
                "custom_model": "GN_CNN_torch_model",
                "custom_model_config": {'feature_dim':128},
            },
            '_disable_preprocessor_api': True,
        }
algo = ppo.PPO(config=config, env='L5-CLE-V1')
algo.restore(checkpoint_path)

2022-12-04 01:13:59,915	WARNING worker.py:1839 -- WARNING: 8 PYTHON worker processes have been started on node: 777dc243b4add8ac085f8950774bd32fe348c2829250d7da57d1a177 with address: 172.28.0.12. This could be a result of using a large number of actors, or due to tasks blocked in ray.get() calls (see https://github.com/ray-project/ray/issues/3644 for some discussion of workarounds).
2022-12-04 01:14:25,297	WARNING catalog.py:641 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2022-12-04 01:14:25,596	INFO trainable.py:164 -- Trainable.setup took 25.769 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-12-04 01:14:25,601	WARNING util.py:66 -- Install gputil for GPU system monitoring.
2022-12-04 01:14:26,933	INFO trainable.py:766 -- Restored on 172.28.0.12 from checkpoint: /content/drive/MyDrive/Colab Notebooks/l5kit/rllib_logs/2022-12-02/

In [ ]:
checkpoint_path =  '/content/drive/MyDrive/Colab Notebooks/l5kit/rllib_logs/'+ str(datetime.date.today())
for i in range(1, 1000):
   # Perform one iteration of training the policy with PPO
   result = algo.train()
   print(pretty_print(result))

   if i % 10 == 0:
       checkpoint = algo.save(checkpoint_dir= checkpoint_path)
       print("checkpoint saved at", checkpoint)

In [ ]:
checkpoint_path =  '/content/drive/MyDrive/Colab Notebooks/l5kit/rllib_logs/'+ str(datetime.date.today())
for i in range(1, 1000):
   # Perform one iteration of training the policy with PPO
   result = algo.train()
   print(pretty_print(result))

   if i % 1 == 0:
       checkpoint = algo.save(checkpoint_dir= checkpoint_path)
       print("checkpoint saved at", checkpoint)

In [ ]:
train_envs = 2
config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 1
# config["framework"] = 'tf2'
config["num_workers"] = 2
config["num_envs_per_worker"] = train_envs
config['_disable_preprocessor_api'] = True,
config["model"]["dim"] = 112
config["model"]["conv_filters"] = [[64, 7, 3], [32, 11, 3], [32, 11, 3]]
config['num_sgd_iter'] = 1
config['sgd_minibatch_size'] = 256
# config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
# config['env_config'] = env_kwargs 
# config["log_level"] = 1
# config["evaluation_interval"] = 1 # change to 10000
# config["evaluation_duration"] = "auto"
# config["evaluation_parallel_to_training"] = True,
# config["evaluation_duration_unit"] = "timesteps"
# config["evaluation_num_workers"] = 3
# config["enable_async_evaluation"] = True,

config["model"]["conv_activation"] = 'relu'
config["model"]["post_fcnet_hiddens"] =  [256]
config["model"]["post_fcnet_activation"] = 'relu'
# config["train_batch_size"] = 200
algo = ppo.PPO(config=config, env="L5-CLE-V1")

2022-12-14 14:57:14,922	WARNING deprecation.py:47 -- DeprecationWarning: `algo = Algorithm(env='L5-CLE-V1', ...)` has been deprecated. Use `algo = AlgorithmConfig().environment('L5-CLE-V1').build()` instead. This will raise an error in the future!
2022-12-14 14:57:14,925	INFO algorithm_config.py:2503 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-12-14 14:57:14,964	INFO algorithm.py:501 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2022-12-14 14:58:13,797	INFO trainable.py:172 -- Trainable.setup took 58.834 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-12-14 14:58:13,800	WARNING util.py:66 -- Install gputil for GPU system monitoring.

In [ ]:
checkpoint_path =  '/content/drive/MyDrive/Colab Notebooks/l5kit/rllib_logs/'+ str(datetime.date.today())
for i in range(1, 1000):
   # Perform one iteration of training the policy with PPO
   result = algo.train()
   print(pretty_print(result))

   if i % 10 == 0:
       checkpoint = algo.save(checkpoint_dir= checkpoint_path)
       print("checkpoint saved at", checkpoint)

## Stable baselines 3

In [ ]:
# Train on episodes of length 32 time steps
train_eps_length = 32
train_envs = 4

# Evaluate on entire scene (~248 time steps)
eval_eps_length = None
eval_envs = 1

# make train env
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = train_eps_length + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
env = make_vec_env("L5-CLE-v0", env_kwargs=env_kwargs, n_envs=train_envs,
                   vec_env_cls=SubprocVecEnv, vec_env_kwargs={"start_method": "fork"})

# make eval env
validation_sim_cfg = SimulationConfigGym()
validation_sim_cfg.num_simulation_steps = None
eval_env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, \
                   'return_info': True, 'train': False, 'sim_cfg': validation_sim_cfg}
eval_env = make_vec_env("L5-CLE-v0", env_kwargs=eval_env_kwargs, n_envs=eval_envs,
                        vec_env_cls=SubprocVecEnv, vec_env_kwargs={"start_method": "fork"})

### Define backbone feature extractor

The backbone feature extractor is shared between the policy and the value networks. The feature extractor *simple_gn* is composed of two convolutional networks followed by a fully connected layer, with ReLU activation. The feature extractor output is passed to both the policy and value networks composed of two fully connected layers with tanh activation (SB3 default).

We perform **group normalization** after every convolutional layer. Empirically, we found that group normalization performs far superior to batch normalization. This can be attributed to the fact that activation statistics change quickly in on-policy algorithms (PPO is on-policy) while batch-norm learnable parameters can be slow to update causing training issues.

In [ ]:
# A simple 2 Layer CNN architecture with group normalization
model_arch = 'simple_gn'
features_dim = 128

# Custom Feature Extractor backbone
policy_kwargs = {
    "features_extractor_class": CustomFeatureExtractor,
    "features_extractor_kwargs": {"features_dim": features_dim, "model_arch": model_arch},
    "normalize_images": False
}

### Clipping Schedule

We linearly decrease the value of the clipping parameter $\epsilon$ as the PPO training progress as it shows improved training stability

In [ ]:
# Clipping schedule of PPO epsilon parameter
start_val = 0.1
end_val = 0.01
training_progress_ratio = 1.0
clip_schedule = get_linear_fn(start_val, end_val, training_progress_ratio)

### Hyperparameters for PPO. 

For detailed description, refer https://stable-baselines3.readthedocs.io/en/master/_modules/stable_baselines3/ppo/ppo.html#PPO

In [ ]:
lr = 3e-4
num_rollout_steps = 256
gamma = 0.8
gae_lambda = 0.9
n_epochs = 10
seed = 42
batch_size = 64
tensorboard_log = '/content/drive/MyDrive/Colab Notebooks/l5kit/tb_logs/' + str(datetime.date.today()) + '/'

### Define the PPO Policy.

SB3 provides an easy interface to the define the PPO policy. Note: We do need to tweak appropriate hyperparameters and the custom policy backbone has been defined above.


In [ ]:
# define model
model = PPO("MultiInputPolicy", env, policy_kwargs=policy_kwargs, verbose=1, n_steps=num_rollout_steps,
            learning_rate=lr, gamma=gamma, tensorboard_log=tensorboard_log, n_epochs=n_epochs,
            clip_range=clip_schedule, batch_size=batch_size, seed=seed, gae_lambda=gae_lambda)

### Defining Callbacks

We can additionally define callbacks to save model checkpoints and evaluate models during training.

In [ ]:
callback_list = []

# Save Model Periodically
save_freq = 10000
save_path = '/content/drive/MyDrive/Colab Notebooks/l5kit/logs/'+ str(datetime.date.today())
output = 'PPO'
checkpoint_callback = CheckpointCallback(save_freq=(save_freq // train_envs), save_path=save_path, \
                                         name_prefix=output)
callback_list.append(checkpoint_callback)

# Eval Model Periodically
eval_freq = 10000
n_eval_episodes = 1
val_eval_callback = L5KitEvalCallback(eval_env, eval_freq=(eval_freq // train_envs), \
                                      n_eval_episodes=n_eval_episodes, n_eval_envs=eval_envs)
callback_list.append(val_eval_callback)


### Train

In [ ]:
outdir = '/content/drive/MyDrive/Colab Notebooks/l5kit/ppo_interrupted/'+ str(datetime.date.today()) + '/'
model_name = 'PPO'
try:
    n_steps = 1000000
    model.learn(n_steps, callback=callback_list)
except:
    model.save(outdir + model_name)
    # model.save_replay_buffer(outdir + model_name+ "_buffer")
    model.policy.save(outdir + model_name + "_policy")

**Voila!** We have a trained PPO policy! Train for larger number of steps for better accuracy. Typical RL algorithms require training atleast 1M steps for good convergence. You can visualize the quantitiative evaluation using tensorboard.

In [ ]:
model = PPO.load('/content/drive/MyDrive/Colab Notebooks/l5kit/ppo_interrupted/PPO2022-11-16.zip', env = env)


In [ ]:
model = PPO.load('/content/drive/MyDrive/Colab Notebooks/l5kit/logs/2022-11-25/PPO_4490000_steps.zip', env = env)
n_steps = 1000000
# model = PPO.load('./PPO_100000_steps.zip', env = env)
model.learn(n_steps, callback=callback_list, reset_num_timesteps=False)


### Visualize the episode from the environment

We can easily visualize the outputs obtained by rolling out episodes in the L5Kit using the Bokeh visualizer.

In [ ]:
model = PPO.load('/content/drive/MyDrive/Colab Notebooks/l5kit/logs/2022-11-23/PPO_4210000_steps.zip', env = env)

In [ ]:
# Train on episodes of length 32 time steps
train_eps_length = 32
train_envs = 4

# Evaluate on entire scene (~248 time steps)
eval_eps_length = None
eval_envs = 1

# make train env
train_sim_cfg = SimulationConfigGym()
train_sim_cfg.num_simulation_steps = 32 + 1
env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, 'sim_cfg': train_sim_cfg}
env = make_vec_env("L5-CLE-v0", env_kwargs=env_kwargs, n_envs=train_envs,
                   vec_env_cls=SubprocVecEnv, vec_env_kwargs={"start_method": "fork"})

# make eval env
validation_sim_cfg = SimulationConfigGym()
validation_sim_cfg.num_simulation_steps = None
eval_env_kwargs = {'env_config_path': env_config_path, 'use_kinematic': True, \
                   'return_info': True, 'train': False, 'sim_cfg': validation_sim_cfg}
eval_env = make_vec_env("L5-CLE-v0", env_kwargs=eval_env_kwargs, n_envs=eval_envs,
                        vec_env_cls=SubprocVecEnv, vec_env_kwargs={"start_method": "fork"})

In [ ]:
rollout_sim_cfg = SimulationConfigGym()
rollout_sim_cfg.num_simulation_steps = None
rollout_env = gym.make("L5-CLE-v0", env_config_path=env_config_path, sim_cfg=rollout_sim_cfg, \
                       use_kinematic=True, train=False, return_info=True)

def rollout_episode(model, env, idx = 0):
    """Rollout a particular scene index and return the simulation output.

    :param model: the RL policy
    :param env: the gym environment
    :param idx: the scene index to be rolled out
    :return: the episode output of the rolled out scene
    """

    # Set the reset_scene_id to 'idx'
    env.reset_scene_id = idx
    
    # Rollout step-by-step
    obs = env.reset()
    done = False
    while True:
        action, _ = model.predict(obs, deterministic=True)
        obs, _, done, info = env.step(action)
        if done:
            break

    # The episode outputs are present in the key "sim_outs"
    sim_out = info["sim_outs"][0]
    return sim_out

# Rollout one episode
# sim_out = rollout_episode(model, rollout_env)
# Rollout 5 episodes
sim_outs =[]
for i in range(5):
    sim_outs.append(rollout_episode(model, rollout_env))

In [ ]:
# might change with different rasterizer
map_API = rollout_env.dataset.rasterizer.sem_rast.mapAPI

def visualize_outputs(sim_outs, map_API):
    for sim_out in sim_outs: # for each scene
        vis_in = episode_out_to_visualizer_scene_gym_cle(sim_out, map_API)
        show(visualize(sim_out.scene_id, vis_in))

output_notebook()
visualize_outputs(sim_outs, map_API)

## Calculate the performance metrics from the episode outputs

We can also calculate the various quantitative metrics on the rolled out episode output. 

In [ ]:
def quantify_outputs(sim_outs, metric_set=None):
    metric_set = metric_set if metric_set is not None else L2DisplacementYawMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE", "ADE"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    tot_ade = 0.0
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        ade_error = scene_metrics["displacement_error_l2"][1:].mean()
        fde_error = scene_metrics['displacement_error_l2'][-1]
        table.add_row([scene_id, round(fde_error.item(), 4), round(ade_error.item(), 4)])
        tot_fde += fde_error.item()
        tot_ade += ade_error.item()

    ave_fde = tot_fde / len(scene_results)
    ave_ade = tot_ade / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4), round(ave_ade, 4)])
    print(table)


quantify_outputs(sim_outs)

+----------+----------+---------+
| scene_id |   FDE    |   ADE   |
+----------+----------+---------+
|    18    | 123.9884 | 58.2833 |
|    76    | 94.0163  | 62.4942 |
|    80    | 95.1473  | 20.5979 |
|    11    | 25.0332  | 18.1672 |
|    16    | 80.5144  |  31.691 |
| Overall  | 83.7399  | 38.2467 |
+----------+----------+---------+


In [ ]:
def quantify_outputs(sim_outs, metric_set=None):
    metric_set = metric_set if metric_set is not None else CLEMetricSet()

    metric_set.evaluate(sim_outs)
    scene_results = metric_set.evaluator.scene_metric_results
    fields = ["scene_id", "FDE", "ADE", "DRT", "CF", "CR", "CS", "PEGO"]
    table = PrettyTable(field_names=fields)
    tot_fde = 0.0
    tot_ade = 0.0
    tot_drt = 0.0
    tot_cf = 0.0
    tot_cr = 0.0
    tot_cs = 0.0
    tot_p_ego = 0.0
    tot_a_ego = 0.0
    # print(scene_results[0])
    for scene_id in scene_results:
        scene_metrics = scene_results[scene_id]
        ade_error = scene_metrics["displacement_error_l2"][1:].mean()
        fde_error = scene_metrics['displacement_error_l2'][-1]
        drt_error = scene_metrics['distance_to_reference_trajectory'][-1]
        cf_error = scene_metrics['collision_front'][-1]
        cr_error = scene_metrics['collision_rear'][-1]
        cs_error = scene_metrics['collision_side'][-1]
        p_ego = scene_metrics['simulated_minus_recorded_ego_speed'][-1]
        # a_ego = scene_metrics['aggressive_ego'][-1]
        table.add_row([scene_id, round(fde_error.item(), 4), round(ade_error.item(), 4), round(drt_error.item(), 4), round(cf_error.item(), 4), round(cr_error.item(), 4), 
        round(cs_error.item(), 4), round(p_ego.item(), 4)])
        tot_fde += fde_error.item()
        tot_ade += ade_error.item()
        tot_drt += drt_error.item()
        tot_cf += cf_error.item()
        tot_cr += cr_error.item()
        tot_cs += cs_error.item()
        tot_p_ego += p_ego.item()
        # tot_a_ego += a_ego.item()

    ave_fde = tot_fde / len(scene_results)
    ave_ade = tot_ade / len(scene_results)
    ave_drt = tot_drt / len(scene_results)
    ave_cf = tot_cf / len(scene_results)
    ave_cr = tot_cr / len(scene_results)
    ave_cs = tot_cs / len(scene_results)
    ave_p_ego = tot_p_ego / len(scene_results)
    # ave_a_ego = tot_a_ego / len(scene_results)
    table.add_row(["Overall", round(ave_fde, 4), round(ave_ade, 4), round(ave_drt, 4), round(ave_cf, 4), round(ave_cr, 4), round(ave_cs, 4), round(ave_p_ego, 4)])
    print(table)


quantify_outputs(sim_outs)

+----------+----------+---------+---------+-----+-----+-----+----------+
| scene_id |   FDE    |   ADE   |   DRT   |  CF |  CR |  CS |   PEGO   |
+----------+----------+---------+---------+-----+-----+-----+----------+
|    18    | 123.9884 | 58.2833 |  14.79  | 0.0 | 0.0 | 0.0 | -3.0587  |
|    76    | 94.0163  | 62.4942 |  4.3439 | 0.0 | 0.0 | 0.0 |  2.7008  |
|    80    | 95.1473  | 20.5979 |  4.7899 | 0.0 | 0.0 | 0.0 | -13.0379 |
|    11    | 25.0332  | 18.1672 |  3.202  | 0.0 | 0.0 | 0.0 |  0.678   |
|    16    | 80.5144  |  31.691 | 22.2307 | 0.0 | 0.0 | 0.0 |  0.138   |
| Overall  | 83.7399  | 38.2467 |  9.8713 | 0.0 | 0.0 | 0.0 |  -2.516  |
+----------+----------+---------+---------+-----+-----+-----+----------+
